In [90]:
import os
import pandas as pd
import numpy as np 
import timeit 

In [ ]:
# Import all raw data from pickled data 


interesting directories: 
Other
Physical Activity 
Sleep 
Stress

Things to explore:
1. how to save the data between uses (so I don't have to read from file every time) 

    pickling? 
    creating a csv? 
    creating a sql database to pull in? 

To do: 
(1) 
keep tables separate until I get a daily summary for each activity 
Tables:
-bpm 
-sleep levels 
-sleep score (done)
-steps
-sedentary minutes (done)
-lightly active minutes (done)
-moderately active minutes (done)
-very active minutes (done)
-resting heartrate (done)
-temperature 
-etc.?
merge all the tables together using date as the primary key 
daily sleep will be the response variabel 
everything else will be explanatory variables 



In [17]:
path = "/Users/jackiekinsler/projects/sleep_analysis_py/physical_data"
 
# to store metric types in a set
unique_metrics = set()

# dirs=directories
for (root, dirs, file) in os.walk(path):
    for f in file:
        if f.endswith(".csv"):
            metric = f.split("-")[0]
            unique_metrics.add(metric)
        
for metric in unique_metrics:
    print(metric)

Respiratory Rate Summary 
Stress Score.csv
Daily SpO2 
Computed Temperature 
sleep_score.csv
Daily Respiratory Rate Summary 
estimated_oxygen_variation
Active Zone Minutes 
Daily Heart Rate Variability Summary 
Device Temperature 
Heart Rate Variability Histogram 
Heart Rate Variability Details 


# IMPORT FROM JSON FUNCTION

In [35]:
# this will streamline importing the many tables from the raw Fitbit JSON 
# Takes a string and returns a dataframe of the concatenated data from the JSON files 
def import_data_from_dir(file_prefix, directory):
    """Reads JSON file(s) in a folder and returns a single dataframe. 
    Takes strings of file_prefix and directory as input. 
    """
    dfs = []
    for file in os.listdir(directory):
        if file_prefix in file: 
            print(file)
            dfs.append(pd.read_json(f"{directory}/{file}"))
    return pd.concat(dfs)

# BPM

In [118]:
# /Users/jackiekinsler/projects/sleep_analysis_py/test_dir
# /Users/jackiekinsler/projects/sleep_analysis_py/physical_data/heart_rate

bpm_nested = import_data_from_dir('heart_rate-', '/Users/jackiekinsler/projects/sleep_analysis_py/physical_data/heart_rate')

heart_rate-2022-03-10.json
heart_rate-2020-08-07.json
heart_rate-2019-06-20.json
heart_rate-2020-06-19.json
heart_rate-2020-04-24.json
heart_rate-2018-12-27.json
heart_rate-2021-10-23.json
heart_rate-2018-02-27.json
heart_rate-2017-08-07.json
heart_rate-2019-09-13.json
heart_rate-2019-05-30.json
heart_rate-2019-10-09.json
heart_rate-2022-07-04.json
heart_rate-2021-08-14.json
heart_rate-2017-10-30.json
heart_rate-2022-04-14.json
heart_rate-2022-06-29.json
heart_rate-2019-03-19.json
heart_rate-2020-03-20.json
heart_rate-2018-05-23.json
heart_rate-2019-02-22.json
heart_rate-2022-07-12.json
heart_rate-2019-12-22.json
heart_rate-2021-04-21.json
heart_rate-2017-10-26.json
heart_rate-2018-06-25.json
heart_rate-2018-04-18.json
heart_rate-2022-08-21.json
heart_rate-2022-04-02.json
heart_rate-2018-07-08.json
heart_rate-2018-09-16.json
heart_rate-2022-03-06.json
heart_rate-2020-08-11.json
heart_rate-2019-08-28.json
heart_rate-2021-02-08.json
heart_rate-2017-08-11.json
heart_rate-2021-12-08.json
h

heart_rate-2018-07-05.json
heart_rate-2021-05-01.json
heart_rate-2022-09-17.json
heart_rate-2022-07-09.json
heart_rate-2019-10-04.json
heart_rate-2021-06-07.json
heart_rate-2018-04-03.json
heart_rate-2018-08-20.json
heart_rate-2020-01-10.json
heart_rate-2019-03-14.json
heart_rate-2019-01-29.json
heart_rate-2022-04-19.json
heart_rate-2022-06-24.json
heart_rate-2020-11-10.json
heart_rate-2019-11-29.json
heart_rate-2021-05-17.json
heart_rate-2017-11-10.json
heart_rate-2022-11-20.json
heart_rate-2020-06-14.json
heart_rate-2020-04-29.json
heart_rate-2019-04-10.json
heart_rate-2022-01-20.json
heart_rate-2021-02-13.json
heart_rate-2021-12-13.json
heart_rate-2018-10-17.json
heart_rate-2020-05-04.json
heart_rate-2021-01-03.json
heart_rate-2018-03-07.json
heart_rate-2017-09-27.json
heart_rate-2021-11-03.json
heart_rate-2017-11-30.json
heart_rate-2021-09-14.json
heart_rate-2020-01-30.json
heart_rate-2019-01-09.json
heart_rate-2022-06-04.json
heart_rate-2020-11-30.json
heart_rate-2019-11-09.json
h

heart_rate-2020-07-24.json
heart_rate-2020-05-19.json
heart_rate-2019-05-20.json
heart_rate-2018-11-27.json
heart_rate-2017-07-24.json
heart_rate-2021-03-23.json
heart_rate-2018-01-27.json
heart_rate-2017-12-31.json
heart_rate-2018-08-11.json
heart_rate-2021-08-28.json
heart_rate-2019-02-08.json
heart_rate-2022-09-26.json
heart_rate-2019-12-08.json
heart_rate-2022-05-05.json
heart_rate-2020-12-31.json
heart_rate-2018-07-22.json
heart_rate-2017-11-21.json
heart_rate-2021-05-26.json
heart_rate-2021-09-05.json
heart_rate-2019-03-25.json
heart_rate-2019-01-18.json
heart_rate-2020-01-21.json
heart_rate-2019-11-18.json
heart_rate-2020-11-21.json
heart_rate-2022-04-28.json
heart_rate-2022-06-15.json
heart_rate-2021-02-22.json
heart_rate-2018-10-26.json
heart_rate-2021-12-22.json
heart_rate-2019-04-21.json
heart_rate-2020-06-25.json
heart_rate-2020-04-18.json
heart_rate-2022-11-11.json
heart_rate-2019-08-02.json
heart_rate-2022-01-11.json
heart_rate-2017-09-16.json
heart_rate-2019-07-31.json
h

heart_rate-2022-04-30.json
heart_rate-2022-08-13.json
heart_rate-2018-09-24.json
heart_rate-2018-05-07.json
heart_rate-2021-07-03.json
heart_rate-2022-01-09.json
heart_rate-2020-08-23.json
heart_rate-2019-06-04.json
heart_rate-2022-11-09.json
heart_rate-2018-12-03.json
heart_rate-2021-10-07.json
heart_rate-2017-08-23.json
heart_rate-2018-02-03.json
heart_rate-2022-02-19.json
heart_rate-2019-07-29.json
heart_rate-2019-05-14.json
heart_rate-2022-10-24.json
heart_rate-2022-12-19.json
heart_rate-2020-07-10.json
heart_rate-2018-11-13.json
heart_rate-2018-01-13.json
heart_rate-2021-03-17.json
heart_rate-2020-04-16.json
heart_rate-2019-06-12.json
heart_rate-2022-03-22.json
heart_rate-2018-02-15.json
heart_rate-2021-10-11.json
heart_rate-2018-12-15.json
heart_rate-2018-10-28.json
heart_rate-2020-07-06.json
heart_rate-2019-05-02.json
heart_rate-2019-09-21.json
heart_rate-2021-03-01.json
heart_rate-2017-09-18.json
heart_rate-2018-01-05.json
heart_rate-2018-11-05.json
heart_rate-2022-09-28.json
h

heart_rate-2018-12-18.json
heart_rate-2018-10-25.json
heart_rate-2021-12-21.json
heart_rate-2018-02-18.json
heart_rate-2021-02-21.json
heart_rate-2022-02-02.json
heart_rate-2022-12-02.json
heart_rate-2018-11-08.json
heart_rate-2018-01-08.json
heart_rate-2017-09-15.json
heart_rate-2021-01-31.json
heart_rate-2020-06-30.json
heart_rate-2022-11-04.json
heart_rate-2019-06-09.json
heart_rate-2022-01-04.json
heart_rate-2019-08-17.json
heart_rate-2022-10-29.json
heart_rate-2022-12-14.json
heart_rate-2020-05-20.json
heart_rate-2019-07-24.json
heart_rate-2019-05-19.json
heart_rate-2022-02-14.json
heart_rate-2021-01-27.json
heart_rate-2017-09-03.json
heart_rate-2018-03-23.json
heart_rate-2021-11-27.json
heart_rate-2020-02-24.json
heart_rate-2022-05-10.json
heart_rate-2020-12-24.json
heart_rate-2019-10-20.json
heart_rate-2021-06-23.json
heart_rate-2018-04-27.json
heart_rate-2017-12-24.json
heart_rate-2017-10-19.json
heart_rate-2018-08-04.json
heart_rate-2020-03-09.json
heart_rate-2019-03-30.json
h

heart_rate-2018-01-15.json
heart_rate-2021-03-11.json
heart_rate-2018-11-15.json
heart_rate-2022-01-19.json
heart_rate-2022-03-24.json
heart_rate-2022-11-19.json
heart_rate-2020-04-10.json
heart_rate-2019-06-14.json
heart_rate-2019-04-29.json
heart_rate-2021-10-17.json
heart_rate-2018-12-13.json
heart_rate-2018-02-13.json
heart_rate-2022-02-09.json
heart_rate-2019-09-27.json
heart_rate-2022-12-09.json
heart_rate-2019-05-04.json
heart_rate-2018-11-03.json
heart_rate-2021-03-07.json
heart_rate-2018-01-03.json
heart_rate-2022-07-30.json
heart_rate-2021-08-20.json
heart_rate-2018-08-19.json
heart_rate-2021-04-03.json
heart_rate-2017-10-04.json
heart_rate-2018-06-07.json
heart_rate-2022-04-20.json
heart_rate-2020-11-29.json
heart_rate-2019-11-10.json
heart_rate-2022-08-03.json
heart_rate-2020-03-14.json
heart_rate-2020-01-29.json
heart_rate-2019-01-10.json
heart_rate-2021-07-13.json
heart_rate-2018-05-17.json
heart_rate-2017-11-29.json
heart_rate-2021-02-06.json
heart_rate-2018-10-02.json
h

heart_rate-2021-12-31.json
heart_rate-2018-12-08.json
heart_rate-2017-08-28.json
heart_rate-2018-02-08.json
heart_rate-2022-02-12.json
heart_rate-2022-12-12.json
heart_rate-2020-05-26.json
heart_rate-2019-07-22.json
heart_rate-2021-11-21.json
heart_rate-2018-11-18.json
heart_rate-2021-01-21.json
heart_rate-2018-03-25.json
heart_rate-2018-01-18.json
heart_rate-2017-09-05.json
heart_rate-2022-05-16.json
heart_rate-2020-12-22.json
heart_rate-2019-10-26.json
heart_rate-2020-02-22.json
heart_rate-2018-08-02.json
heart_rate-2021-06-25.json
heart_rate-2021-04-18.json
heart_rate-2017-12-22.json
heart_rate-2018-04-21.json
heart_rate-2022-06-06.json
heart_rate-2022-08-18.json
heart_rate-2021-09-16.json
heart_rate-2021-07-08.json
heart_rate-2018-07-31.json


In [121]:
bpm_nested.to_pickle("bpm_nested.pkl")
# Index needs to be reset as there is no primary key 
bpm_nested.reset_index(inplace = True)

In [153]:
bpm_explode = pd.json_normalize(bpm_nested['value'])

In [156]:
bpm_explode.head()

,bpm,confidence
0,54,3
1,53,2
2,53,3
3,54,3
4,54,3


In [157]:
# Here, two columns are brought together: the dateTime column from bpm_nested, 
# and the two exploded columns (bpm, confidence) that makeup bpm_explode 
bpm = pd.concat([bpm_nested['dateTime'], bpm_explode], axis = 1)

In [158]:
bpm.head()

,dateTime,bpm,confidence
0,2022-03-10 08:00:08,54,3
1,2022-03-10 08:00:13,53,2
2,2022-03-10 08:00:18,53,3
3,2022-03-10 08:00:33,54,3
4,2022-03-10 08:00:48,54,3


In [159]:
bpm.to_pickle("bpm.pkl")

In [148]:
detail = bpm_nested.loc[[0,1,2,3]]
detail_explode = pd.json_normalize(detail['value'])
detail_explode

,bpm,confidence
0,54,3
1,53,2
2,53,3
3,54,3


In [160]:
test = pd.read_pickle("bpm.pkl")

From the Fitbit community: 
“Confidence scores indicate confidence in the accuracy of the captured data with “3” being the highest and “0” being the lowest; a score of 0 indicates no Heart Rate is detected by the Fitbit tracker and no Heart Rate is displayed on the tracker device”.

# SLEEP DETAIL

In [115]:
# Importing detailed sleep data 
df_sleep_nested = import_data_from_dir('sleep-', '/Users/jackiekinsler/projects/sleep_analysis_py/test_dir')

df_sleep_nested.head()
df_sleep_nested.loc[0,'levels']
# the summary of the sleep data seems to have everything interesting (cumulative data)
# the rest of the levels column is super detailed information about each type and duration of sleep segment 

sleep-2021-04-28.json
sleep-2018-07-13.json


0    {'summary': {'deep': {'count': 7, 'minutes': 7...
0    {'summary': {'deep': {'count': 5, 'minutes': 9...
Name: levels, dtype: object

In [69]:
df_sleep_nested.head()

,logId,dateOfSleep,startTime,endTime,duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,efficiency,type,infoCode,logType,levels,mainSleep
0,19156244739,2018-08-12,2018-08-11T23:58:00.000,2018-08-12T08:04:30.000,29160000,0,428,58,0,486,91,stages,0,auto_detected,"{'summary': {'deep': {'count': 5, 'minutes': 9...",True
1,19146597672,2018-08-11,2018-08-10T23:06:00.000,2018-08-11T08:19:00.000,33180000,0,455,98,0,553,88,stages,0,auto_detected,"{'summary': {'deep': {'count': 7, 'minutes': 1...",True
2,19140164672,2018-08-10,2018-08-09T23:23:00.000,2018-08-10T08:38:30.000,33300000,0,473,82,0,555,90,stages,0,auto_detected,"{'summary': {'deep': {'count': 7, 'minutes': 1...",True
3,19129203312,2018-08-09,2018-08-08T23:34:00.000,2018-08-09T07:35:00.000,28860000,0,414,67,0,481,96,stages,0,auto_detected,"{'summary': {'deep': {'count': 6, 'minutes': 8...",True
4,19122684524,2018-08-08,2018-08-07T22:07:00.000,2018-08-08T06:00:00.000,28380000,0,419,54,0,473,94,stages,0,auto_detected,"{'summary': {'deep': {'count': 5, 'minutes': 9...",True


In [67]:
df_sleep_nested['dateOfSleep'].is_unique
df_sleep_nested['dateOfSleep'].duplicated
# below I'm grabbing the data from a night that had two sleep entries (aka a duplicate date)
df_sleep_nested.loc[df_sleep_nested['dateOfSleep'] == '2018-07-31']

,logId,dateOfSleep,startTime,endTime,duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,efficiency,type,infoCode,logType,levels,mainSleep
11,19036267747,2018-07-31,2018-07-31T06:07:00.000,2018-07-31T07:41:00.000,5640000,0,85,9,0,94,90,classic,2,auto_detected,"{'summary': {'restless': {'count': 4, 'minutes...",False
12,19033727531,2018-07-31,2018-07-30T23:35:00.000,2018-07-31T05:01:30.000,19560000,0,282,44,4,326,92,stages,0,auto_detected,"{'summary': {'deep': {'count': 4, 'minutes': 6...",True


sleep score in fitbit: 
source: https://help.fitbit.com/articles/en_US/Help_article/2439.htm



Note: shortData is only included for stages sleep logs and includes wake periods that are 3 minutes or less in duration. This distinction is to simplify graphically distinguishing short wakes from longer wakes, but they are physiologically equivalent.

from: https://dev.fitbit.com/build/reference/web-api/sleep/get-sleep-log-by-date-range/#:~:text=Note%3A%20shortData%20is%20only%20included,but%20they%20are%20physiologically%20equivalent.

sleep score is in a .csv with seemingly 1 row per day 

# SLEEP SCORE

In [72]:
# Importing sleep score from .csv 
# /Users/jackiekinsler/projects/sleep_analysis_py/JacklynKinsler/Sleep/sleep_score.csv
sleep_score = pd.read_csv('/Users/jackiekinsler/projects/sleep_analysis_py/JacklynKinsler/Sleep/sleep_score.csv')
sleep_score.head()

,sleep_log_entry_id,timestamp,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness
0,39497287745,2022-12-19T11:00:30Z,86,22,22,42,102,53,0.080976
1,39483974879,2022-12-18T07:17:00Z,84,21,21,42,105,53,0.080969
2,39483974878,2022-12-17T02:48:30Z,61,15,18,28,53,52,0.095041
3,39457704744,2022-12-16T04:45:00Z,67,17,19,31,74,52,0.103937
4,39447333347,2022-12-15T08:07:30Z,77,19,21,37,77,51,0.093642


import steps data 
steps data is recorded every few minutes, and it records the steps in that period of time 
will reduce the data to get the total steps for each day 

# STEPS

In [116]:
# Importing step data 

step_detail = import_data_from_dir('steps-', '/Users/jackiekinsler/projects/sleep_analysis_py/test_dir')
step_detail.head()

steps-2017-09-16.json
steps-2017-10-16.json


,dateTime,value
0,2017-09-16 09:59:00,0
1,2017-09-16 11:23:00,0
2,2017-09-16 11:24:00,16
3,2017-09-16 11:25:00,0
4,2017-09-16 11:26:00,0


In [117]:
step_detail.loc[step_detail['value'] != 0]

,dateTime,value
2,2017-09-16 11:24:00,16
15,2017-09-16 11:37:00,41
21,2017-09-16 11:43:00,8
22,2017-09-16 11:44:00,2
28,2017-09-16 11:50:00,6
...,...,...
39133,2017-11-15 03:08:00,31
39140,2017-11-15 03:15:00,4
39145,2017-11-15 03:20:00,7
39159,2017-11-15 03:34:00,14


# ACTIVITY MINUTES

In [34]:
# Importing activity minutes 

sedentary_minutes = import_data_from_dir('sedentary_minutes', '/Users/jackiekinsler/projects/sleep_analysis_py/physical_data/Physical_Activity')
lightly_active = import_data_from_dir('lightly_active', '/Users/jackiekinsler/projects/sleep_analysis_py/physical_data/Physical_Activity')
moderately_active = import_data_from_dir('moderately_active', '/Users/jackiekinsler/projects/sleep_analysis_py/physical_data/Physical_Activity')
very_active = import_data_from_dir('very_active_minutes', '/Users/jackiekinsler/projects/sleep_analysis_py/physical_data/Physical_Activity')

# Sort ascending by date
sedentary_minutes.sort_values(by='dateTime', inplace=True)
lightly_active.sort_values(by='dateTime', inplace=True)
moderately_active.sort_values(by='dateTime', inplace=True)
very_active.sort_values(by='dateTime', inplace=True)

# Pickle the data for future use 
sedentary_minutes.to_pickle("sedentary_minutes.pkl")
lightly_active.to_pickle("lightly_active.pkl")
moderately_active.to_pickle("moderately_active.pkl")
very_active.to_pickle("very_active.pkl")

sedentary_minutes-2021-11-24.json
sedentary_minutes-2019-02-08.json
sedentary_minutes-2017-09-16.json
sedentary_minutes-2017-08-17.json
sedentary_minutes-2021-10-25.json
sedentary_minutes-2020-08-01.json
sedentary_minutes-2021-01-28.json
sedentary_minutes-2022-07-22.json
sedentary_minutes-2017-10-16.json
sedentary_minutes-2021-09-25.json
sedentary_minutes-2020-05-03.json
sedentary_minutes-2022-06-22.json
sedentary_minutes-2018-10-11.json
sedentary_minutes-2019-12-05.json
sedentary_minutes-2020-04-03.json
sedentary_minutes-2018-11-10.json
sedentary_minutes-2018-09-11.json
sedentary_minutes-2022-10-20.json
sedentary_minutes-2018-06-13.json
sedentary_minutes-2020-07-02.json
sedentary_minutes-2020-08-31.json
sedentary_minutes-2018-04-14.json
sedentary_minutes-2022-05-23.json
sedentary_minutes-2021-03-29.json
sedentary_minutes-2020-09-30.json
sedentary_minutes-2017-07-18.json
sedentary_minutes-2022-09-20.json
sedentary_minutes-2021-12-24.json
sedentary_minutes-2022-08-21.json
sedentary_minu

very_active_minutes-2018-12-10.json
very_active_minutes-2022-04-23.json
very_active_minutes-2018-05-14.json
very_active_minutes-2020-06-02.json
very_active_minutes-2018-07-13.json
very_active_minutes-2018-04-14.json
very_active_minutes-2022-05-23.json
very_active_minutes-2021-03-29.json
very_active_minutes-2020-09-30.json
very_active_minutes-2020-08-31.json
very_active_minutes-2020-07-02.json
very_active_minutes-2022-10-20.json
very_active_minutes-2018-06-13.json
very_active_minutes-2021-07-27.json
very_active_minutes-2020-10-30.json
very_active_minutes-2022-08-21.json
very_active_minutes-2022-09-20.json
very_active_minutes-2021-12-24.json
very_active_minutes-2017-07-18.json
very_active_minutes-2019-09-06.json
very_active_minutes-2019-05-09.json
very_active_minutes-2021-02-27.json
very_active_minutes-2019-08-07.json
very_active_minutes-2019-10-06.json
very_active_minutes-2020-01-04.json
very_active_minutes-2018-01-14.json
very_active_minutes-2018-02-13.json
very_active_minutes-2022-03-

# RESTING HEARTRATE

In [106]:
# Importing resting heartrate data 
resting_heartrate_nested = import_data_from_dir('resting_heart_rate', '/Users/jackiekinsler/projects/sleep_analysis_py/physical_data/Physical_Activity')
# The data in this dataframe is nested, and only the last column ('value') has the needed data 
resting_heartrate = pd.json_normalize(resting_heartrate_nested['value'])
# 'date' comes in as a string, convert it to datetime 
resting_heartrate['date'] = pd.to_datetime(resting_heartrate['date'])
resting_heartrate.sort_values(by='date', inplace=True)
resting_heartrate.to_pickle("very_active.pkl")

resting_heart_rate-2022-07-17.json
resting_heart_rate-2019-07-18.json
resting_heart_rate-2020-07-17.json
resting_heart_rate-2017-07-18.json
resting_heart_rate-2018-07-18.json
resting_heart_rate-2021-07-17.json
